In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# ! pip install googletrans
# ! pip install jieba
! pip install fasttext

     |████████████████████████████████| 71kB 2.1MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3015700 sha256=4ef9ce1cf78e6d486c4805dfd40327aed98158d1a05c0238e917768694c5e6d9
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
import numpy as np
from numpy import array, argmax, random, take 
import pandas as pd
import nltk
# from googletrans import Translator
import string 
import jieba
import re 

# from keras.models import Sequential 
# from keras.layers import Dense, LSTM, Embedding, RepeatVector
# from keras.preprocessing.text import Tokenizer
# from keras.callbacks import ModelCheckpoint 
# from keras.preprocessing.sequence import pad_sequences
# from keras.models import load_model 
# from keras import optimizers 
import matplotlib.pyplot as plt 

from tqdm import tqdm

tqdm.pandas()

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [ ]:
# script to keep colab from disconnecting 
function ClickConnect(){
    console.log("Clicked on connect button"); 
    document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect,60000)

# script to notify when done running
import smtplib
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login("dominique.a.sy@gmail.com", "")
msg = "COLAB WORK FINISH ALERT!"
server.sendmail("dominique.a.sy@gmail.com", "dominique.a.sy@gmail.com", msg)
server.quit()

# to kill the kernel
!kill -9 -1

# Loading

In [ ]:
path = "/content/drive/My Drive/COLAB NOTEBOOKS/SCL/C04"
train_en=pd.read_csv(path+'/train_en.csv')
train_tcn=pd.read_csv(path+'/train_tcn.csv')
test=pd.read_csv(path+'/test_tcn.csv')
dev_en=pd.read_csv(path+'/dev_en.csv')
dev_tcn=pd.read_csv(path+'/dev_tcn.csv')

In [ ]:
train_en.head()

,product_title,category
0,Recollections Color Splash Clear Stamps & Stencil,Hobbies & Stationery
1,"soap,lotion scrub set 400",Health & Personal Care
2,Spigen Galaxy S10e Case Tough Armor Gunmetal,Mobile Accessories
3,Acrylic Lanalon Bright Red,Hobbies & Stationery
4,303 FLAT SHEET/Blanket 100% cotton,Home & Living


In [ ]:
train_tcn.head()

,product_title,category
0,Gucci Gucci Guilty Pour Femme Stud Edition 罪愛女...,Health & Beauty
1,（二手）PS4 GTA 5 俠盜獵車手5 Grand Theif Auto V繁體 中文版,Game Kingdom
2,百獸卡,Life & Entertainment
3,nac nac活氧全效柔衣素,Mother & Baby
4,#Nike耐吉官方F.C. 男子足球長褲新款標準型 拒水 拉鏈褲腳\nCD0557,Men's Apparel


In [ ]:
train_en.shape

(500000, 2)

In [ ]:
train_tcn.shape

(500000, 2)

# Google Translate

In [ ]:
translator=Translator()
translator.detect(train_tcn.product_title[0]).lang

'zh-CN'

In [ ]:
translator.translate(train_tcn.iloc[0,0], dest='en', src='zh-CN').text

'Gucci Gucci Guilty Pour Femme Stud Edition Guilty Eau de Toilette Limited Edition 50ml T'

In [ ]:
sample=train_tcn.product_title[:1000].progress_apply(lambda x: translator.translate(x, dest='en', src='zh-CN').text)

100%|██████████| 1000/1000 [03:05<00:00,  5.39it/s]


In [ ]:
sample.head()

0      Gucci Gucci Guilty Pour Femme Stud Edition Gui...
1      (Pre-owned) PS4 GTA5 Grand Theft Auto 5 Grand ...
2                                                    百獸卡
3                      nac nac active oxygen full effect
4      #NikeNike official F.C. men's football trouser...
                             ...                        
995                                  中大童 粉色 馬卡龍色系點點甜美 洋裝
996    Nike basketball shoes women's shoes Jordan 7th...
997                                          NS-樂高-侏羅紀世界
998     全新正品天梭運動PRS200計時石英防水日曆商務休閒鋼帶男錶T067.417.11.051.01
999                                       長榮哩程 轉讓 17500哩
Name: product_title, Length: 1000, dtype: object

# Unsupervised Machine Translation

In [ ]:
train_tcn['cut_cols'] = train_tcn['product_title'].apply(lambda x: jieba.lcut(str(x)))
# train_tcn.to_csv(path+'/train_cut.csv', index=False)
train_tcn['cut_cols']

In [ ]:
train_tcn=pd.read_csv(path+'/train_cut.csv')

In [ ]:
# tokenized words in the vocabulary
slist =[]
for x in train_tcn['cut_cols']:
    slist.extend(x)

In [ ]:
len(slist)

8599189

In [ ]:
len(set(slist))

481694

In [ ]:
# removing punctuations
ch_list = []
# stopwords = [ line.rstrip() for line in open(path+'/stop_words.txt',"r", encoding="utf-8")]
stopwords = [ line.rstrip() for line in open(path+'/punc.txt',"r", encoding="utf-8")]
for seg in set(slist):
      if seg not in stopwords:
          ch_list.append(seg)

In [ ]:
len(ch_list)

480935

In [ ]:
import fasttext
import fasttext.util

fasttext.util.download_model('zh', if_exists='ignore') 
ft = fasttext.load_model('cc.zh.300.bin')

In [ ]:
inter = set(ft.get_labels()).intersection(set(slist))

In [ ]:
diff = set(slist)-set(ft.get_labels())

In [ ]:
diff[:10]

In [ ]:
# Words that have embeddings
inter

{'臥式',
 '包斯',
 '何潤東',
 '刮破',
 '輪刀',
 'chains',
 'RiDATA',
 '籌碼',
 '林雅萍',
 'Impeccable',
 '2cm',
 'Awesome',
 '江南',
 '晶片',
 'Walls',
 '近幾年',
 '中下層',
 '4780',
 'IPhone6',
 '組加',
 '頭彩',
 'polo',
 '糖霜',
 '左營區',
 '雄一',
 '牌雙',
 '減慢',
 'smith',
 '齒數',
 '白蘿卜',
 '硅酮',
 '附卡',
 '小卉',
 '原件',
 'mmm',
 '氣度',
 '統計',
 '塵世',
 'N90',
 '慶慶',
 '農場裡',
 '購買處',
 '看視',
 '不等式',
 '請購',
 '28.6',
 '白嫩',
 '花花',
 '扣車',
 '賣會',
 '齊心',
 '1246',
 '女心',
 '享免',
 '鑽飾品',
 '秤子',
 '噴灑',
 'TTO',
 '求財',
 '配布',
 '王君',
 'Marionette',
 'sonny',
 '當孕',
 '冷感',
 'portrait',
 'PKP',
 '鎖針',
 '翻書',
 'Coco',
 '撢',
 '煤爐',
 '內稽',
 '週計',
 '裘',
 '套間',
 '8026',
 '阪本',
 'Unichi',
 '天章',
 '李飛',
 'INTENSIVE',
 '號室',
 '恩重',
 '波佐見燒',
 '明細',
 '黑絲',
 'Verilog',
 '呢帽',
 '幣',
 '本遊戲',
 '2003.9',
 'Kiki',
 '數字時',
 'hsu',
 'Vince',
 '五結鄉',
 '炊',
 'Grenelle',
 'MERCY',
 '高機動',
 '行動電',
 'a',
 '悠居',
 'Weave',
 '乾衣機',
 'bee',
 'LO',
 '魚店',
 '小破',
 'Tucker',
 'Taek',
 '往昔',
 '斗羅',
 '經典橘',
 '雜牌',
 '腺',
 'DESIRE',
 '粉頰',
 'CHOCOOLATE',
 '滋润',
 '媽媽們',
 'ROX',
 

In [ ]:
train_tcn.dropna(subset=['product_title'], inplace=True)
train_tcn[train_tcn['product_title'].str.contains('코')]

In [ ]:
list(inter)[:10]

['臥式', '包斯', '何潤東', '刮破', '輪刀', 'chains', 'RiDATA', '籌碼', '林雅萍', 'Impeccable']

In [ ]:
zh_embeddings = pd.DataFrame(columns=['labels','vector'])

In [ ]:
# word vector for each word in the vocabulary
emb = []
count = 0 
for i in list(inter):
  count+=1
  emb.append(ft.get_word_vector(i))
  if count%1000==0:
    print(".", end="")

.........................................................................................................................................................

In [ ]:
zh_embeddings['labels'] = pd.Series(list(inter))
zh_embeddings['vector'] = pd.Series(emb)

In [ ]:
zh_embeddings['labels'].nunique()

153620

In [ ]:
zh_embeddings.head()

,labels,vector
0,臥式,"[0.4088176, 0.02333656, 0.03943201, 0.06168215..."
1,包斯,"[-0.06076532, 0.07766768, -0.04050674, -0.0049..."
2,何潤東,"[0.046263136, -0.084913686, 0.049385455, 0.132..."
3,刮破,"[0.1107896, 0.15822938, 0.17494443, 0.07522982..."
4,輪刀,"[0.015036436, -0.059545863, 0.03072809, 0.1581..."


In [ ]:
zh_embeddings.to_csv('zh_embeddings.csv', index=False)

# XLM-R


In [ ]:
import torch
! pip install sentencepiece
xlmr = torch.hub.load('pytorch/fairseq', 'xlmr.large')
xlmr.eval()
tokens=xlmr.encode(train_tcn['cut_cols'])
tokens

In [ ]:
xlmr.extract_features(tokens)

tensor([[[-0.0084,  0.0306,  0.1065,  ..., -0.1425, -0.0022, -0.0485],
         [-0.5199, -0.0722, -0.0239,  ...,  0.1168,  0.1225, -0.4428],
         [-0.3858,  0.0979,  0.0305,  ...,  0.2274, -0.0807, -0.0345],
         ...,
         [-0.1731,  0.1132, -0.0336,  ...,  0.0114, -0.4444,  0.0751],
         [ 0.1929,  0.0952, -0.0347,  ..., -0.0358, -0.3404,  0.0285],
         [ 0.1057,  0.0129,  0.0227,  ...,  0.0135, -0.0372,  0.0198]]],
       grad_fn=<TransposeBackward0>)